In [18]:
#https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
import pandas as pd

data = pd.read_csv('datasets/abcnews-date-text.csv');
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [34]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem as stemmer
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to /home/juanjo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [36]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []

for word in doc_sample.split(' '):
    words.append(word)
    
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [37]:
print(len(documents))
print(documents[:5])

1082168
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [38]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [39]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0

for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [40]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [41]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4015, 1)]

In [42]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


In [52]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5913144510648105),
 (1, 0.38522584492986706),
 (2, 0.49651004561935946),
 (3, 0.5053969162540006)]


In [53]:
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [45]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.030*"queensland" + 0.023*"perth" + 0.023*"market" + 0.019*"jail" + 0.019*"hospit" + 0.018*"china" + 0.016*"share" + 0.015*"power" + 0.013*"year" + 0.013*"donald"
Topic: 1 
Words: 0.054*"australia" + 0.021*"countri" + 0.019*"rural" + 0.017*"hour" + 0.017*"live" + 0.015*"nation" + 0.015*"west" + 0.012*"busi" + 0.012*"peopl" + 0.011*"farmer"
Topic: 2 
Words: 0.056*"polic" + 0.025*"death" + 0.023*"sydney" + 0.022*"attack" + 0.020*"crash" + 0.020*"woman" + 0.017*"die" + 0.014*"shoot" + 0.013*"investig" + 0.013*"arrest"
Topic: 3 
Words: 0.031*"south" + 0.023*"north" + 0.015*"kill" + 0.014*"park" + 0.014*"protest" + 0.013*"close" + 0.011*"build" + 0.011*"program" + 0.011*"coast" + 0.010*"young"
Topic: 4 
Words: 0.031*"govern" + 0.019*"council" + 0.017*"say" + 0.017*"plan" + 0.014*"rise" + 0.014*"water" + 0.012*"concern" + 0.011*"industri" + 0.009*"polit" + 0.009*"resid"
Topic: 5 
Words: 0.021*"adelaid" + 0.018*"open" + 0.016*"australian" + 0.015*"women" + 0.015*"final" + 0.

In [46]:
# Running LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"govern" + 0.008*"health" + 0.008*"fund" + 0.007*"plan" + 0.007*"council" + 0.006*"elect" + 0.006*"say" + 0.005*"mine" + 0.005*"budget" + 0.005*"hill"
Topic: 1 Word: 0.026*"countri" + 0.024*"hour" + 0.018*"market" + 0.011*"share" + 0.008*"busi" + 0.006*"australian" + 0.006*"august" + 0.006*"syria" + 0.006*"islam" + 0.006*"wall"
Topic: 2 Word: 0.009*"live" + 0.009*"royal" + 0.008*"commiss" + 0.007*"trade" + 0.006*"australia" + 0.006*"dairi" + 0.006*"obama" + 0.006*"scott" + 0.005*"white" + 0.005*"china"
Topic: 3 Word: 0.009*"david" + 0.008*"monday" + 0.008*"june" + 0.007*"open" + 0.006*"toni" + 0.006*"updat" + 0.006*"abbott" + 0.006*"social" + 0.005*"extend" + 0.005*"quiz"
Topic: 4 Word: 0.019*"polic" + 0.016*"charg" + 0.012*"crash" + 0.012*"woman" + 0.011*"interview" + 0.010*"kill" + 0.010*"shoot" + 0.009*"attack" + 0.009*"arrest" + 0.008*"murder"
Topic: 5 Word: 0.011*"podcast" + 0.009*"australia" + 0.009*"leagu" + 0.008*"final" + 0.008*"world" + 0.008*"sport" + 0.

In [55]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [47]:
# Performance evaluation by classifying sample document using LDA Bag of Words model
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41980382800102234	 
Topic: 0.035*"court" + 0.024*"charg" + 0.023*"face" + 0.022*"murder" + 0.018*"accus" + 0.016*"child" + 0.016*"trial" + 0.014*"abus" + 0.013*"guilti" + 0.013*"tell"

Score: 0.2200627624988556	 
Topic: 0.021*"school" + 0.018*"interview" + 0.015*"labor" + 0.014*"indigen" + 0.013*"children" + 0.013*"fund" + 0.013*"life" + 0.012*"student" + 0.012*"commun" + 0.012*"miss"

Score: 0.22005252540111542	 
Topic: 0.031*"south" + 0.023*"north" + 0.015*"kill" + 0.014*"park" + 0.014*"protest" + 0.013*"close" + 0.011*"build" + 0.011*"program" + 0.011*"coast" + 0.010*"young"

Score: 0.02001926489174366	 
Topic: 0.021*"adelaid" + 0.018*"open" + 0.016*"australian" + 0.015*"women" + 0.015*"final" + 0.015*"brisban" + 0.015*"melbourn" + 0.014*"world" + 0.012*"gold" + 0.011*"win"

Score: 0.020015599206089973	 
Topic: 0.032*"elect" + 0.018*"hous" + 0.016*"chang" + 0.016*"price" + 0.015*"canberra" + 0.014*"lose" + 0.013*"tasmanian" + 0.013*"health" + 0.013*"farm" + 0.011*"public"


In [48]:
# Performance evaluation by classifying sample document using LDA TF-IDF model
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199660778045654	 
Topic: 0.008*"price" + 0.008*"turnbul" + 0.006*"climat" + 0.006*"bushfir" + 0.006*"flood" + 0.006*"wednesday" + 0.006*"cattl" + 0.006*"christma" + 0.006*"rain" + 0.006*"hous"

Score: 0.02000577002763748	 
Topic: 0.029*"rural" + 0.012*"news" + 0.009*"violenc" + 0.008*"nation" + 0.007*"thursday" + 0.007*"domest" + 0.006*"cancer" + 0.006*"april" + 0.005*"disabl" + 0.005*"research"

Score: 0.02000419795513153	 
Topic: 0.011*"govern" + 0.008*"health" + 0.008*"fund" + 0.007*"plan" + 0.007*"council" + 0.006*"elect" + 0.006*"say" + 0.005*"mine" + 0.005*"budget" + 0.005*"hill"

Score: 0.020003696903586388	 
Topic: 0.024*"trump" + 0.011*"weather" + 0.009*"michael" + 0.009*"septemb" + 0.007*"fiji" + 0.007*"rugbi" + 0.006*"explain" + 0.006*"breakfast" + 0.006*"wild" + 0.006*"shorten"

Score: 0.020003532990813255	 
Topic: 0.019*"polic" + 0.016*"charg" + 0.012*"crash" + 0.012*"woman" + 0.011*"interview" + 0.010*"kill" + 0.010*"shoot" + 0.009*"attack" + 0.009*"arrest" + 0

In [49]:
# Testing model on unseen document
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8499764800071716	 Topic: 0.030*"queensland" + 0.023*"perth" + 0.023*"market" + 0.019*"jail" + 0.019*"hospit"
Score: 0.016669662669301033	 Topic: 0.031*"south" + 0.023*"north" + 0.015*"kill" + 0.014*"park" + 0.014*"protest"
Score: 0.01666959933936596	 Topic: 0.031*"govern" + 0.019*"council" + 0.017*"say" + 0.017*"plan" + 0.014*"rise"
Score: 0.016669509932398796	 Topic: 0.028*"trump" + 0.015*"tasmania" + 0.013*"record" + 0.012*"time" + 0.012*"break"
Score: 0.016669489443302155	 Topic: 0.032*"elect" + 0.018*"hous" + 0.016*"chang" + 0.016*"price" + 0.015*"canberra"
Score: 0.016669269651174545	 Topic: 0.054*"australia" + 0.021*"countri" + 0.019*"rural" + 0.017*"hour" + 0.017*"live"
Score: 0.01666918210685253	 Topic: 0.021*"school" + 0.018*"interview" + 0.015*"labor" + 0.014*"indigen" + 0.013*"children"
Score: 0.016668954864144325	 Topic: 0.056*"polic" + 0.025*"death" + 0.023*"sydney" + 0.022*"attack" + 0.020*"crash"
Score: 0.016668954864144325	 Topic: 0.021*"adelaid" + 0.018*"open"